In [151]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [49]:
data = pd.read_csv('D:\Maskininlärning\WineQT.csv')

In [50]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


In [51]:
data = data.drop('Id', axis =1)

In [52]:
X = data.drop('quality', axis=1)# Separera input attribut och target 
y = data['quality']

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [54]:
forest = RandomForestClassifier(random_state=42)

In [55]:
forest.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [56]:
train_data_score = forest.score(X_train, y_train)
test_data_score = forest.score(X_test, y_test)

In [57]:
print('Modellen får testscore:', test_data_score.round(2))

Modellen får testscore: 0.67


In [60]:
n_trees = len(forest.estimators_)
print('Det finns', n_trees,'träd i skogen.')

Det finns 100 träd i skogen.


In [58]:
importances = forest.feature_importances_
attributes = X.columns
most_important_attribut = ''
most_important_importance = 0
for feature, importance in zip(attributes, importances):
    if importance > most_important_importance:
        most_important_attribut = feature
        most_important_importance = importance
        
print(f"Störst påverkan har {most_important_attribut} som är {most_important_importance.round(2)}")

Störst påverkan har alcohol som är 0.15


In [73]:
import warnings
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)
difficult_classes = []
report = classification_report(y_test, y_pred, output_dict=True)
for label in y_test.unique():
    if report[str(label)]['f1-score'] < 0.5:
        difficult_classes.append(label)
print('Modellen har svårt att klassificera rätt på följande klasser:', difficult_classes)

Modellen har svårt att klassificera rätt på följande klasser: [7, 4, 8]


### Genom att göra en classification_report så kan man se att dom med lägst precision eller recall är dom klasser som modellen har störst problem med att klassificera rätt. så i detta fall vill jag säga 4, 7 och 8

### I följande implementation tränar vi en uppsättning av beslutsträd, där varje träd är begränsad av parametrarna max_depth, min_samples_split och min_samples_leaf. Vi använder också bootstrap sampling för att skapa olika träningsmängder för varje träd.

In [116]:
class RandomForest:
    def __init__(self, n_trees= None, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.trees = []
    
    def fit(self, X, y):
        for i in range(self.n_trees):
            tree = DecisionTreeClassifier(max_depth=self.max_depth, 
                                          min_samples_split=self.min_samples_split,
                                          min_samples_leaf=self.min_samples_leaf)
            indices = random.sample(range(len(X)), len(X))
            X_train = X.iloc[indices]
            y_train = y.iloc[indices]
            tree.fit(X_train, y_train)
            self.trees.append(tree)
    
    def predict(self, X):
        y_preds = []
        for tree in self.trees:
            y_pred = tree.predict(X)
            y_preds.append(y_pred)
        y_preds = np.array(y_preds)
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=y_preds)

### För att använda vår egen Random Forest kan vi göra följande:

In [148]:
forest = RandomForest(50)
forest.fit(X_train, y_train)

In [149]:
train_data_score = accuracy_score(y_train, forest.predict(X_train))
test_data_score = accuracy_score(y_test, forest.predict(X_test))
print('Modellen får testscore:', test_data_score.round(2))
n_trees = len(forest.trees)
print('Det finns', n_trees,'träd i skogen.')

Modellen får testscore: 0.58
Det finns 50 träd i skogen.


In [150]:
report = classification_report(y_test, forest.predict(X_test))
print(report)

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         0
           4       0.10      0.09      0.10        11
           5       0.67      0.69      0.68       148
           6       0.59      0.52      0.55       145
           7       0.43      0.54      0.48        37
           8       0.00      0.00      0.00         2

    accuracy                           0.58       343
   macro avg       0.30      0.31      0.30       343
weighted avg       0.59      0.58      0.58       343



### Modellen har svårt att klassificera rätt på klasserna 3, 4, och 8. Det är inte samma klasser som modellen i godkänt delen hade svårt med.

### Det är möjligt att den egna implementationen av Random Forest-skogen skiljer sig från modellen i godkänt delen på grund av att antalet beslutsträd och andra parametrar kan ha olika värden.